In [38]:
import numpy as np
import json

np_cards = np.array(json.load(open('data/cards.json', 'r', encoding='utf-8')))

In [39]:
def text_preprocess(input_text):
    # remove all stop words
    input_text = ' '.join([word for word in input_text.split() if word not in stopwords])

    input_text = ''.join([char for char in input_text if char.isalnum() or char == '/' or char == ' '])

    return input_text

In [40]:
import spacy

en = spacy.load('en_core_web_sm')
stopwords = en.Defaults.stop_words

color_identity_letter = ['w', 'u', 'b', 'r', 'g', 'c']

inputs = []
# input text = type_line + rules_text (if not None) + power / toughness (if not None)
for card in np_cards:
    if sum(card['color_identity']) > 1:
        continue

    input_text = card['type_line']
    if card['rules_text'] is not None:
        input_text += '\n' + card['rules_text']
    if card['power'] is not None:
        input_text += '\n' + card['power'] + '/' + card['toughness']

    input_text = text_preprocess(input_text)

    # index 0 is w, 1 is u, 2 is b, 3 is r, 4 is g, 5 is c
    # convert color identity to corresponding letter
    target = ''
    for i in range(len(card['color_identity'])):
        if card['color_identity'][i] == 1:
            target = color_identity_letter[i]
            break
    inputs.append((input_text, target))

print(inputs[10])

('Enchantment  Aura Enchant creature When CARDNAME enters battlefield tap enchanted creature Enchanted creature doesnt untap controllers untap step', 'u')


In [41]:
# train test split
from sklearn.model_selection import train_test_split

train, test = train_test_split(inputs, test_size=0.2, random_state=42)

In [42]:
#tfidf
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer()
x_train = tfidf.fit_transform([x[0] for x in train])
x_test = tfidf.transform([x[0] for x in test])

y_train = [x[1] for x in train]
y_test = [x[1] for x in test]

# predict
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(max_iter=1000)
clf.fit(x_train, y_train)
print(clf.score(x_test, y_test))

0.7605131860299359


In [43]:
test_text = 'Instant\nYou gain 3 life.'

test_text = text_preprocess(test_text)

test_text = tfidf.transform([test_text])
print(clf.predict(test_text))

['w']
